In [1]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint

from sheet_id.models.FCN import FCN
from sheet_id.utils.loss_functions import softmax_sparse_crossentropy_ignoring_background
from sheet_id.utils.metrics import sparse_accuracy_ignoring_background
from sheet_id.utils.dataPreprocessing import splitTrainValidation
from sheet_id.utils.dataGenerator import DataGenerator

/home/mirlab/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = FCN(input_shape=(250,250,1), n_classes=124)

In [3]:
loss_fn = softmax_sparse_crossentropy_ignoring_background
optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, 
                 epsilon=1e-8, decay=0.0, amsgrad=False)
metrics = [sparse_accuracy_ignoring_background]
model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)

In [4]:
callbacks = []

In [5]:
# checkpoint = ModelCheckpoint(filepath='./checkpoints/checkpoint_weights.hdf5', 
#                              save_weights_only=True, period=5) #.{epoch:d}
# callbacks.append(checkpoint)

In [6]:
train_partition, validation_partition = splitTrainValidation('../data/', max_size=500, test_size=20, npy_only=True)

In [7]:
params = {'dim': (250,250),
          'batch_size': 20,
          'n_classes': 124,
          'n_channels': 1,
          'shuffle': True,
          'crop': True,
          'crop_size': (250,250),
          'load_npy': True}
training_generator = DataGenerator(train_partition, **params)
validation_generator = DataGenerator(validation_partition, **params)

In [8]:
history = model.fit_generator(generator=training_generator, epochs=30,
                              validation_data=validation_generator,
                              use_multiprocessing=True, workers=10,
                              callbacks=callbacks)

Epoch 1/30
24/24 [==============================] - 500s 21s/step - loss: 0.0512 - sparse_accuracy_ignoring_background: 0.4178 - val_loss: 0.0283 - val_sparse_accuracy_ignoring_background: 0.4610
Epoch 2/30
24/24 [==============================] - 251s 10s/step - loss: 0.0375 - sparse_accuracy_ignoring_background: 0.5286 - val_loss: 0.0453 - val_sparse_accuracy_ignoring_background: 0.5394
Epoch 3/30
24/24 [==============================] - 503s 21s/step - loss: 0.0261 - sparse_accuracy_ignoring_background: 0.5875 - val_loss: 0.0227 - val_sparse_accuracy_ignoring_background: 0.4662
Epoch 4/30
24/24 [==============================] - 250s 10s/step - loss: 0.0211 - sparse_accuracy_ignoring_background: 0.6434 - val_loss: 0.0213 - val_sparse_accuracy_ignoring_background: 0.6016
Epoch 5/30
24/24 [==============================] - 536s 22s/step - loss: 0.0212 - sparse_accuracy_ignoring_background: 0.6030 - val_loss: 0.0328 - val_sparse_accuracy_ignoring_background: 0.5351


OSError: Unable to create file (unable to open file: name = './checkpoints/checkpoint_weights.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
example_img, example_annotation = training_generator[0]

In [ ]:
model.evaluate(example_img, example_annotation)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
prediction = model.predict(example_img)

In [ ]:
prediction = np.argmax(prediction, axis=-1)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,3,1)
plt.imshow(example_img[0,:,:,0], cmap='gray')
plt.subplot(1,3,2)
plt.imshow(example_annotation[0,:,:,0], cmap='gray')
plt.subplot(1,3,3)
plt.imshow(prediction[0,:,:], cmap='gray')

In [ ]:
np.unique(example_annotation[0,:,:,0])

In [ ]:
np.unique(prediction)

In [ ]:
cl = 3
vis_ref = np.zeros_like(example_annotation[0,:,:,0])
vis = np.zeros_like(example_annotation[0,:,:,0])
for row in range(vis.shape[0]):
    for col in range(vis.shape[1]):
        if example_annotation[0,row,col,0] == cl:
            vis_ref[row,col] = 255
        if prediction[0, row, col] == cl:
            vis[row,col] = 255

plt.subplot(1,2,1)            
plt.imshow(vis_ref, cmap='gray')
plt.subplot(1,2,2)            
plt.imshow(vis, cmap='gray')

print(vis)